In [6]:
import scanpy as sc
import anndata as ad
import muon as mu
import mudata as md
import numpy as np
import pandas as pd
import loompy as lp
import os
import warnings
import scipy
import celltypist
import random
import re

In [2]:
mu.set_options(pull_on_update = False)

In [3]:
day_14_data_combined = mu.read_h5mu('D14_CITESeq_ALL.h5mu')
day_28_data_combined = mu.read_h5mu('D28_CITESeq_ALL.h5mu')

## Create NK specific mudata objects

In [4]:
# ==========================================
# 1. Process Day 14 NK Subset
# ==========================================

# Remove the old neighbor graph from the parent object.
# It causes the "incorrect shape" crash and we don't need it for the subset.
for key in list(day_14_data_combined.obsp.keys()):
    del day_14_data_combined.obsp[key]

# Also clear the uns['neighbors'] if it exists, just to be clean
if 'neighbors' in day_14_data_combined.uns:
    del day_14_data_combined.uns['neighbors']

# Define clusters to pull in all NK clusters
clusters_to_keep = ['NK', 'pNK']

#Name of the obs columns we are subsetting on
obs_col = 'celltype' 

# NOW Subset
D14_NK = day_14_data_combined[day_14_data_combined.obs[obs_col].isin(clusters_to_keep)].copy()

print(f"✅ Subsetting successful! D14 NK cells: {D14_NK.n_obs}")

# ---------------------------------------------------------
# Now continue with re-analysis...
# ---------------------------------------------------------

# Recompute RNA
sc.pp.pca(D14_NK.mod['rna'], svd_solver='arpack', random_state=123)
sc.pp.neighbors(D14_NK.mod['rna'], n_neighbors=10, random_state=123)
sc.tl.leiden(D14_NK.mod['rna'], resolution=0.5, random_state=123, key_added='leiden_reculstered')
sc.tl.umap(D14_NK.mod['rna'], random_state=123)

# Recompute Protein
sc.pp.pca(D14_NK.mod['prot'], svd_solver='arpack', random_state=123)
sc.pp.neighbors(D14_NK.mod['prot'], n_neighbors=10, random_state=123)
sc.tl.leiden(D14_NK.mod['prot'], resolution=0.5, random_state=123, key_added='leiden_reculstered')
sc.tl.umap(D14_NK.mod['prot'], random_state=123)

# Recompute Global (WNN)
mu.pp.neighbors(D14_NK, n_neighbors=10, random_state=123)
mu.tl.leiden(D14_NK, resolution=0.5, random_state=123, key_added='leiden_reculstered')
mu.tl.umap(D14_NK, random_state=123)



# ==========================================
# 2. Process Day 28 NK Subset
# ==========================================
# Remove the old neighbor graph from the parent object.
# It causes the "incorrect shape" crash and we don't need it for the subset.
for key in list(day_28_data_combined.obsp.keys()):
    del day_28_data_combined.obsp[key]

# Also clear the uns['neighbors'] if it exists, just to be clean
if 'neighbors' in day_28_data_combined.uns:
    del day_28_data_combined.uns['neighbors']

# Define clusters to pull in all NK clusters
clusters_to_keep = ['NK', 'pNK']

#Name of the obs columns we are subsetting on
obs_col = 'celltype' 

# NOW Subset
D28_NK = day_28_data_combined[day_28_data_combined.obs[obs_col].isin(clusters_to_keep)].copy()

print(f"✅ Subsetting successful! D28 NK cells: {D28_NK.n_obs}")

# ---------------------------------------------------------
# Now continue with re-analysis...
# ---------------------------------------------------------

# Recompute RNA
sc.pp.pca(D28_NK.mod['rna'], svd_solver='arpack', random_state=123)
sc.pp.neighbors(D28_NK.mod['rna'], n_neighbors=10, random_state=123)
sc.tl.leiden(D28_NK.mod['rna'], resolution=0.5, random_state=123, key_added='leiden_reculstered')
sc.tl.umap(D28_NK.mod['rna'], random_state=123)

# Recompute Protein
sc.pp.pca(D28_NK.mod['prot'], svd_solver='arpack', random_state=123)
sc.pp.neighbors(D28_NK.mod['prot'], n_neighbors=10, random_state=123)
sc.tl.leiden(D28_NK.mod['prot'], resolution=0.5, random_state=123, key_added='leiden_reculstered')
sc.tl.umap(D28_NK.mod['prot'], random_state=123)

# Recompute Global (WNN)
mu.pp.neighbors(D28_NK, n_neighbors=10, random_state=123)
mu.tl.leiden(D28_NK, resolution=0.5, random_state=123, key_added='leiden_reculstered')
mu.tl.umap(D28_NK, random_state=123)

✅ Subsetting successful! D14 NK cells: 573
✅ Subsetting successful! D28 NK cells: 2729


Generate a column that subsets the NK by CD16 (Stage 4 and Stage 5)

In [5]:
#Day 14 data
#Seperating by CD16+ and Negative
# 1. Grab the expression values (handles sparse/dense/layers automatically)
vals = sc.get.obs_df(D14_NK['prot'], keys=['CD16_TotalSeqC'])

# 2. Create the new column using np.where
# Syntax: np.where(condition, value_if_true, value_if_false)
D14_NK['prot'].obs['CD16_status'] = np.where(vals['CD16_TotalSeqC'] >= 10, 'S5', 'S4')
D14_NK['rna'].obs['CD16_status'] = D14_NK['prot'].obs['CD16_status']
D14_NK['rna'].obs["CD16_cell_condition"] = np.where(
    D14_NK['rna'].obs["cell_condition"].str.contains("pNK", regex=True),
    D14_NK['rna'].obs["cell_condition"],  # use only cell_condition
    D14_NK['rna'].obs["CD16_status"].astype(str) + "_" + D14_NK['rna'].obs["cell_condition"].astype(str)
)

#Day 28 Data
#Seperating by CD16+ and Negative
# 1. Grab the expression values (handles sparse/dense/layers automatically)
vals = sc.get.obs_df(D28_NK['prot'], keys=['CD16_TotalSeqC'])

# 2. Create the new column using np.where
# Syntax: np.where(condition, value_if_true, value_if_false)
D28_NK['prot'].obs['CD16_status'] = np.where(vals['CD16_TotalSeqC'] >= 8, 'S5', 'S4')
D28_NK['rna'].obs['CD16_status'] = D28_NK['prot'].obs['CD16_status']
D28_NK['rna'].obs["CD16_cell_condition"] = np.where(
    D28_NK['rna'].obs["cell_condition"].str.contains("pNK", regex=True),
    D28_NK['rna'].obs["cell_condition"],  # use only cell_condition
    D28_NK['rna'].obs["CD16_status"].astype(str) + "_" + D28_NK['rna'].obs["cell_condition"].astype(str)
)

## NK speciic pyscenic run

In [8]:
adatas = [D14_NK, D28_NK]
names = ["D14_NK", "D28_NK"]
#os.mkdir('pyscenic') #remove hash to make the directory that will save the loom files

for ad, name in zip(adatas, names):
    ad['rna'].X = ad['rna'].layers['counts'].copy()
    
    adata_row_attrs = {
        'Gene': np.array(ad['rna'].var_names)
        }
    adata_col_attrs = {
        'CellID': np.array(ad['rna'].obs_names),
        'nGene': np.array(np.sum(ad['rna'].X.transpose() > 0, axis=0)).flatten(),
        'nUMI': np.array(np.sum(ad['rna'].X.transpose(), axis=0)).flatten()
        }
    
    lp.create(f'pyscenic/{name}.loom', ad['rna'].X.transpose(), adata_row_attrs, adata_col_attrs)

In [9]:
%%writefile pyscenic/NK_run_pyscenic_job.sh
#!/bin/bash
#SBATCH --account=PAS2527
#SBATCH --time=2:00:00
#SBATCH --mail-type=ALL
#SBATCH --ntasks-per-node=80
#SBATCH --partition=nextgen
#SBATCH --output=CITEseq_NK_pyscenic.slurm-%j.out
#SBATCH --error=CITEseq_NK_pyscenic.slurm-%j.err

# Exit on error
set -e

# Load modules and activate conda environment with pyscenic
module load miniconda3/24.1.2-py310
source activate pyscenic

# ==============================================================================
# CONFIGURATION
# ==============================================================================
# Define the source directory to make the script cleaner
SRC_DIR="$HOME/Single_Cell_Files/03012023_CITESeq/scanpy_muon/pyscenic"
DB_DIR="$HOME/Single_Cell_Files/03012023_CITESeq/scanpy_muon/pyscenic/pySCENIC_files"

# Copy Static Database Files to $TMPDIR (Do this once)
echo "Copying reference databases to $TMPDIR..."
rsync -av $DB_DIR/allTFs_hg38.txt $TMPDIR/
rsync -av $DB_DIR/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl $TMPDIR/
rsync -av $DB_DIR/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather $TMPDIR/
rsync -av $DB_DIR/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather $TMPDIR/

# Move to scratch
cd $TMPDIR

# ==============================================================================
# FUNCTION: Run Pipeline for a specific Sample
# ==============================================================================
run_pyscenic() {
    SAMPLE_ID=$1
    echo "----------------------------------------------------------------"
    echo "STARTING PIPELINE FOR: $SAMPLE_ID"
    echo "----------------------------------------------------------------"

    # 1. Copy specific loom file to TMP
    echo "Copying $SAMPLE_ID.loom..."
    rsync -av $SRC_DIR/$SAMPLE_ID.loom $TMPDIR/

    # 2. GRN Step
    echo "Running GRN for $SAMPLE_ID..."
    pyscenic grn \
        -o ${SAMPLE_ID}_adj.csv \
        ${SAMPLE_ID}.loom allTFs_hg38.txt \
        --num_workers 80 \
        --seed 123

    # 3. CTX Step
    # Note: Outputting as .csv (standard), assumed previously .yml in your script
    echo "Running CTX for $SAMPLE_ID..."
    pyscenic ctx \
        -o ${SAMPLE_ID}_reg.csv \
        --expression_mtx_fname ${SAMPLE_ID}.loom \
        --annotations_fname motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl \
        ${SAMPLE_ID}_adj.csv \
        hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \
        hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \
        --num_workers 40 \
        --mode custom_multiprocessing \
        --mask_dropouts

    # 4. AUCell Step
    echo "Running AUCell for $SAMPLE_ID..."
    pyscenic aucell \
        -o ${SAMPLE_ID}_pyscenic_results.loom \
        ${SAMPLE_ID}.loom ${SAMPLE_ID}_reg.csv \
        --num_workers 40 \
        --seed 123

    # 5. Copy Results Back
    echo "Copying results for $SAMPLE_ID to a results folder"
    OUT_FOLDER="$SRC_DIR/${SAMPLE_ID}_results"
    mkdir -p $OUT_FOLDER
    rsync -av ${SAMPLE_ID}_adj.csv $OUT_FOLDER
    rsync -av ${SAMPLE_ID}_reg.csv $OUT_FOLDER
    rsync -av ${SAMPLE_ID}_pyscenic_results.loom $OUT_FOLDER
    
    # Clean up loom from TMP to save space for next run
    rm ${SAMPLE_ID}.loom
    rm ${SAMPLE_ID}_adj.csv
    rm ${SAMPLE_ID}_reg.csv
    rm ${SAMPLE_ID}_pyscenic_results.loom
    
    echo "Completed $SAMPLE_ID"
}

# ==============================================================================
# EXECUTION
# ==============================================================================

# Run for D28
run_pyscenic "D28_NK"

# Run for D14
run_pyscenic "D14_NK"

echo "All samples processed successfully!"

Writing pyscenic/NK_run_pyscenic_job.sh


In [8]:
# ==============================================================================
# CONFIGURATION
# ==============================================================================
# Update this to wherever your "pyscenic" folder lives relative to this notebook
PYSCENIC_DIR = 'pyscenic' 

# ==============================================================================
# FUNCTION: Import PySCENIC Results
# ==============================================================================
def import_pyscenic_results(mdata, sample_id):
    """
    Loads AUCell matrix from Loom, cleans columns, aligns to MuData, and saves to .obsm
    """
    # 1. Construct the path based on your Bash script's folder structure
    #    Path: pyscenic/D28_results/D28_pyscenic_results.loom
    loom_path = os.path.join(PYSCENIC_DIR, f"{sample_id}_results", f"{sample_id}_pyscenic_results.loom")
    
    if not os.path.exists(loom_path):
        print(f"❌ Error: File not found: {loom_path}")
        return

    print(f"Loading results for {sample_id} from: {loom_path}")
    
    # 2. Connect and load data
    with lp.connect(loom_path, mode='r', validate=False) as lf:
        auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)

    # 3. Clean the column names (RegEx)
    #    Turns "Regulon(SOX2(+))" -> "SOX2"
    new_columns = [re.sub(r"Regulon\((\w+)\(.*\)\)", r"\1", c) for c in auc_mtx.columns]
    auc_mtx.columns = new_columns

    # 4. Re-index to match the specific MuData object
    #    This ensures cells are in the exact same order as your mdata object
    #    and fills missing cells with NaN (though there shouldn't be any if indices match)
    auc_mtx = auc_mtx.reindex(mdata['rna'].obs_names)

    # 5. Save to .obsm
    mdata['rna'].obsm['NK_Pyscenic_AUC'] = auc_mtx

    print(f"✅ Success! Added {auc_mtx.shape[1]} regulons to .obsm['NK_Pyscenic_AUC']")
    print("-" * 40)

# ==============================================================================
# EXECUTION
# ==============================================================================

# Run for Day 14
import_pyscenic_results(D14_NK, "D14_NK")

# Run for Day 28
import_pyscenic_results(D28_NK, "D28_NK")

Loading results for D14_NK from: pyscenic/D14_NK_results/D14_NK_pyscenic_results.loom
✅ Success! Added 273 regulons to .obsm['NK_Pyscenic_AUC']
----------------------------------------
Loading results for D28_NK from: pyscenic/D28_NK_results/D28_NK_pyscenic_results.loom
✅ Success! Added 219 regulons to .obsm['NK_Pyscenic_AUC']
----------------------------------------


## Module Scoring based off of tonsil NKDIs

In [6]:
#Import the tonsil and blood anndata objects generated from Li et al Cell Reports 2023 to capture the NKDI scoring at a single cell level
tonsil_ad = sc.read_h5ad('/users/PAS1800/ruesch6/Single_Cell_Files/pySCENIC_Analyses/tonsil_ad.h5ad')
blood_ad = sc.read_h5ad('/users/PAS1800/ruesch6/Single_Cell_Files/Anderson_Blood_NK_RNAseq/Scanpy/singlet_adata.h5ad')

In [7]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning, message=".*fragmented.*")
    sc.tl.rank_genes_groups(tonsil_ad, groupby='cell_type', method='wilcoxon', corr_method='benjamini-hochberg', layer='log_counts')
    sc.tl.rank_genes_groups(tonsil_ad, groupby='leiden',group='28', method='wilcoxon', corr_method='benjamini-hochberg', layer='log_counts', key_added='pNK')
    sc.tl.rank_genes_groups(blood_ad, groupby='cell_type', method='wilcoxon', corr_method='benjamini-hochberg', layer='log_counts')

In [8]:
cells = ['Stage3', 'Stage4a', 'Stage4b', 'Stage5', 'pNK', 'CD56_bright', 'CD56_dim', 'Proliferating_NK']
cell_dict = {}
for cell in cells:
    if cell == 'pNK':
        x = sc.get.rank_genes_groups_df(tonsil_ad, group='28', log2fc_min=0, pval_cutoff=0.05, key=cell)
        genes = x['names'].tolist()[0:50]
        cell_dict[cell] = genes
    elif cell in ['CD56_bright', 'CD56_dim', 'Proliferating_NK']:
        x = sc.get.rank_genes_groups_df(blood_ad, group=cell, log2fc_min=0, pval_cutoff=0.05)
        genes = x['names'].tolist()[0:50]
        cell_dict[cell] = genes
    else:
        x = sc.get.rank_genes_groups_df(tonsil_ad, group=cell, log2fc_min=0, pval_cutoff=0.05)
        genes = x['names'].tolist()[0:50]
        cell_dict[cell] = genes

In [9]:
for cell in cells:
    sc.tl.score_genes(D14_NK['rna'], gene_list=cell_dict[cell], layer='log_counts', score_name=f'{cell}_score')
    sc.tl.score_genes(D28_NK['rna'], gene_list=cell_dict[cell], layer='log_counts', score_name=f'{cell}_score')

## Saving
Save the NK specific mudata objects as well

In [9]:
D14_NK.write_h5mu('D14_CITESeq_NK.h5mu', compression = 'lzf')
D28_NK.write_h5mu('D28_CITESeq_NK.h5mu', compression = 'lzf')

## Session Info

In [10]:
import session_info
session_info.show(excludes=['distributed'])